In [83]:
import uproot
import numpy as np
import awkward as ak
#import pandas as pd
#import matplotlib.pyplot as plt

In [84]:
data_path = '../data/NANOAOD_2017C.root'

data = uproot.open(f'{data_path}:Events;1')

In [90]:
data_length = len(data['event'].array())


target_hlt_keys = [
    'HLT_Tau3Mu_Mu5_Mu1_TkMu1_IsoTau10_Charge1',
    'HLT_Tau3Mu_Mu7_Mu1_TkMu1_IsoTau15_Charge1',
    'HLT_Tau3Mu_Mu7_Mu1_TkMu1_Tau15_Charge1',
]

data_hlt_keys = data.keys(filter_name='HLT_Tau3Mu*')

hlt_keys = [hlt_key for hlt_key in target_hlt_keys if hlt_key in data_hlt_keys]

print(f"hlt_keys: {hlt_keys}")

hlt_mask = ak.Array(np.zeros(data_length, dtype=bool))

for hlt_key in hlt_keys:
    hlt_mask = hlt_mask | data[hlt_key].array()


print(f"Before filtering, Event #: {data_length}")
print(f"After filtering, Event #: {np.count_nonzero(hlt_mask)}")

hlt_keys: ['HLT_Tau3Mu_Mu5_Mu1_TkMu1_IsoTau10_Charge1']
Before filtering, Event #: 1615379
After filtering, Event #: 122521


In [106]:
muon_keys = [
    'nMuon',
    'Muon_pt',
    'Muon_eta',
    'Muon_phi',
    'Muon_charge',
    'Muon_mass',
]

tau_keys = [
    'nTau',
    'Tau_pt',
    'Tau_eta',
    'Tau_phi',
    'Tau_charge',
    'Tau_mass',
]

keys = muon_keys + tau_keys

data_ak = data.arrays(keys)
print(len(data_ak))

data_ak = data_ak[hlt_mask]
print(len(data_ak))

print(data_ak)
print(data_ak[0])

1615379
122521
[{nMuon: 2, Muon_pt: [6.05, 4.87], Muon_eta: [-1.68, ...], ...}, ..., {...}]
{nMuon: 2, Muon_pt: [6.05, 4.87], Muon_eta: [-1.68, ...], Muon_phi: [...], ...}


In [107]:
print(data_ak['Muon_pt'])

muon_mask = data_ak['Muon_pt'] > 5
print(muon_mask)


[[6.05, 4.87], [6.38, 3.93], [11.2, ..., 7.84], ..., [19, 5.52], [6.98, 4.25]]
[[True, False], [True, False], [True, ...], ..., [True, True], [True, False]]


In [81]:
muon_mask = (data_ak['Muon_pt'] > 1) & (np.abs(data_ak['Muon_eta']) < 4)
print(muon_mask)

### HLT masking 처럼 앞쪽에서 array로 뽑아낸걸 이용해서 filter 만들고 걸러내야할거같은데..?

[[True, True], [True, True], [True, ..., True], ..., [True, True], [True, True]]
